In [ ]:
pip install pytesseract pillow nltk spacy && sudo apt install -y tesseract-ocr || brew install tesseract || (Invoke-WebRequest -Uri "https://github.com/UB-Mannheim/tesseract/releases/latest/download/tesseract-ocr-w64-setup.exe" -OutFile "tesseract.exe" && Start-Process "tesseract.exe" -Wait) && python -c "import nltk; nltk.download('punkt'); nltk.download('stopwords')" && python -m spacy download pt_core_news_sm

In [4]:
import pytesseract
from PIL import Image
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
import spacy

# Carregar modelo de linguagem do SpaCy
nlp = spacy.load("en_core_web_sm")

# Baixar as stopwords do NLTK
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Função para realizar OCR em uma imagem
def perform_ocr(image_path):
    # Abrir a imagem
    image = Image.open(image_path)
    # Realizar OCR usando Tesseract
    text = pytesseract.image_to_string(image)
    return text

# Função para realizar a sumarização de texto
def perform_summarization(text):
    # Tokenize o texto em frases, divide o texto em frases usando a função sent_tokenize da biblioteca NLTK, considerando o idioma ingles padrao.
    sentences = sent_tokenize(text)
    # Tokenize as palavras, divide o texto em palavras usando a função word_tokenize da biblioteca NLTK.
    words = word_tokenize(text)
    # Remova stopwords, remove as palavras irrelevantes (stop words) do texto.
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word.lower() not in stop_words]
    # Calcule a frequência das palavras, calcula a frequência das palavras no texto usando a classe FreqDist da biblioteca NLTK.
    freq_dist = FreqDist(words)
    # Calcule a frequência máxima, calcula a frequência máxima entre todas as palavras no texto.
    max_freq = max(freq_dist.values())
    # Normalizar as frequências, normaliza as frequências das palavras dividindo a frequência de cada palavra pela frequência máxima.
    for word in freq_dist.keys():
        freq_dist[word] = (freq_dist[word]/max_freq)
    # Calcule a pontuação das frases
    sentence_scores = {}
    for sentence in sentences:
        #Divide a frase em palavras, converte para minúsculas
        for word in word_tokenize(sentence.lower(), language='english'):  
            #Verifica se a palavra está presente nas palavras relevantes (que não são stop words).
            if word in freq_dist.keys():
                #Verifica se a frase tem menos de 30 palavras.
                if len(sentence.split(' ')) < 30: 
                    #Verifica se a frase não está presente no dicionário de pontuações das frases.
                    if sentence not in sentence_scores.keys(): 
                        #Atribui a pontuação da palavra à frase no dicionário de pontuações das frases.
                        sentence_scores[sentence] = freq_dist[word] 
                    #Se a frase já estiver presente no dicionário de pontuações das frases.
                    else:
                        #Incrementa a pontuação da frase com a pontuação da palavra.
                        sentence_scores[sentence] += freq_dist[word] 
    # Obtenha as frases mais importantes
    #Ordena as frases com base em suas pontuações, do maior para o menor, e seleciona as três primeiras frases.
    summarized_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:3]
    #Une as frases selecionadas em um único resumo, separadas por espaço.
    summary = ' '.join(summarized_sentences) 
    return summary

# Caminho da imagem
image_path = "texto.png"

# Realizar OCR
print("Performing OCR...")
ocr_text = perform_ocr(image_path)
print("OCR Result:")
print(ocr_text)
print()

# Realizar sumarização
print("Performing Summarization...")
summary_text = perform_summarization(ocr_text)
print("Summary:")
print(summary_text)
print()



ModuleNotFoundError: No module named 'spacy'